Logistic regression
In this project, I implemented the cost function and gradient descent
Some parts I do not understand very well, I just follow the instructor and do so, but I can't do it very fluently by myself

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = 'ex2data1.txt'
data = pd.read_csv(path, header = None, names = ['Exam 1', 'Exam 2', 'Admitted'])
data.head()

,Exam 1,Exam 2,Admitted
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1


In [3]:
#sigmoid function
def sigmoid(z):
    return 1/(1 + np.exp(-z))

In [4]:
#cost function
def cost(theta, X, y):
    #X, theta, y還沒有轉到matrix形式,他們還是array
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    first = np.multiply(-y,np.log(sigmoid(X*theta.T)))  #注意h(x) = sigmoid(X*theta.T); linear的hypothesis是theta.T*X,其實邏輯回歸就是將綫性的結果投影到sigmoid函數裏
    second = np.multiply((1-y), np.log(1 - sigmoid(X*theta.T)))

    return np.sum(first - second)/len(X)    #記得用sum來做求和，否則只是各項對應相乘


In [5]:
#convert data into X and y

data.insert(0,'Ones',1)

#切割數據
cols = data.shape[1]
X = data.iloc[:,0:cols-1]
y = data.iloc[:,cols-1:cols]

#convert to array
X = np.array(X.values)
y = np.array(y.values)
theta = np.zeros(3)

In [6]:
#gradient descent
def gradient(theta, X, y):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    paramaters = int(theta.ravel().shape[1])
    grad = np.zeros(paramaters)

    error = sigmoid(X*theta.T) - y

    for i in range(paramaters):
        term = np.multiply(error,X[:,i])
        grad[i] = np.sum(term)/len(X)

    return grad

In [7]:
X.shape, theta.shape, y.shape

((100, 3), (3,), (100, 1))

In [8]:
cost(theta, X, y)

0.6931471805599453

In [9]:
gradient(theta, X, y)

array([ -0.1       , -12.00921659, -11.26284221])

In [10]:
import scipy.optimize as opt
result = opt.fmin_tnc(func=cost, x0=theta, fprime=gradient, args=(X, y))
result

(array([-25.16131872,   0.20623159,   0.20147149]), 36, 0)

In [11]:
#predict function, the final model
def predict(theta, X):
    probability = sigmoid(X*theta.T)
    return[1 if x > 0.5 else 0 for x in probability]

In [12]:
#do not understand this part very well
#not implemented by me
theta_min = np.matrix(result[0])
predictions = predict(theta_min, X)
correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(predictions, y)]
accuracy = (sum(map(int, correct)) % len(correct))
print ('accuracy = {0}%'.format(accuracy))

accuracy = 89%


This is the regularized logistic regression

In [13]:
#regularized cost function
def costReg(theta, X, y, learningRate):
    X = np.matrix(X)
    theta = np.matrix(theta)
    y = np.matrix(y)

    first = np.multiply(-y, np.log(sigmoid(X*theta.T)))
    second = np.multiply(1-y, np.log(1-sigmoid(X*theta.T)))

    reg = (learningRate/(2*len(X))) * np.sum(np.power(theta[:,1:theta.shape[1]],2)) #不理解theta[:,theta.shape[1]] Ans: 他這裏是去掉第一個theta後面所有的theta，記得第一個theta為截距，不用被正規化
    return np.sum(first - second) / len(X) + reg

In [14]:
def gradientReg(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    paramaters = int(theta.ravel().shape[1])
    grad = np.zeros(paramaters)
    error = sigmoid(X*theta.T) - y

    for i in range(paramaters):
        term = np.multiply(error,X[:,i])
        if i == 0:
            grad[i] = np.sum(term)/len(X)
        else:
            grad[i] = np.sum(term)/len(X) + (learningRate/len(X))*theta[:,i]
            #Question：爲什麽沒有減法，不是theta = theta - 一坨東西嗎？Ans 這個function返回的只是梯度
    return grad

In [15]:
path =  'ex2data2.txt'
data2 = pd.read_csv(path, header=None, names=['Test 1', 'Test 2', 'Accepted'])
data2.head()

,Test 1,Test 2,Accepted
0,0.051267,0.69956,1
1,-0.092742,0.68494,1
2,-0.213710,0.69225,1
3,-0.375000,0.50219,1
4,-0.513250,0.46564,1


In [16]:
degree = 5
x1 = data2['Test 1']
x2 = data2['Test 2']

data2.insert(3, 'Ones', 1)

#generate polynomial
for i in range(1, degree):
    for j in range(0, i):
        data2['F' + str(i) + str(j)] = np.power(x1, i-j) * np.power(x2, j)

data2.drop('Test 1', axis=1, inplace=True)
data2.drop('Test 2', axis=1, inplace=True)

data2.head()

,Accepted,Ones,F10,F20,F21,F30,F31,F32,F40,F41,F42,F43
0,1,1,0.051267,0.002628,0.035864,0.000135,0.001839,0.025089,0.000007,0.000094,0.001286,0.017551
1,1,1,-0.092742,0.008601,-0.063523,-0.000798,0.005891,-0.043509,0.000074,-0.000546,0.004035,-0.029801
2,1,1,-0.213710,0.045672,-0.147941,-0.009761,0.031616,-0.102412,0.002086,-0.006757,0.021886,-0.070895
3,1,1,-0.375000,0.140625,-0.188321,-0.052734,0.070620,-0.094573,0.019775,-0.026483,0.035465,-0.047494
4,1,1,-0.513250,0.263426,-0.238990,-0.135203,0.122661,-0.111283,0.069393,-0.062956,0.057116,-0.051818


In [17]:
# set X and y (remember from above that we moved the label to column 0)
cols = data2.shape[1]
X2 = data2.iloc[:,1:cols]
y2 = data2.iloc[:,0:1]

# convert to numpy arrays and initalize the parameter array theta
X2 = np.array(X2.values)
y2 = np.array(y2.values)
theta2 = np.zeros(11)

In [18]:
X2.shape, theta2.shape, y2.shape

((118, 11), (11,), (118, 1))

In [19]:
learningRate = 1

In [20]:
costReg(theta2, X2, y2, learningRate)

0.6931471805599454

In [21]:
gradientReg(theta2, X2, y2, learningRate)

array([0.00847458, 0.01878809, 0.05034464, 0.01150133, 0.01835599,
       0.00732393, 0.00819244, 0.03934862, 0.00223924, 0.01286005,
       0.00309594])

In [22]:
#not implemented by me
result2 = opt.fmin_tnc(func=costReg, x0=theta2, fprime=gradientReg, args=(X2, y2, learningRate))
result2

(array([ 0.53010248,  0.29075567, -1.60725764, -0.58213819,  0.01781027,
        -0.21329508, -0.40024142, -1.37144139,  0.02264304, -0.9503358 ,
         0.0344085 ]),
 22,
 1)

In [23]:
#do not understand this part very well
#not implemented by me
theta_min = np.matrix(result2[0])
predictions = predict(theta_min, X2)
correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(predictions, y2)]
accuracy = (sum(map(int, correct)) / len(correct))
print ('accuracy = {0}%'.format(accuracy))

accuracy = 0.6610169491525424%
